# Velocity Autocorrelation Function (VACF) and Spectrum Calculation in Liquid Argon

This notebook simulates a three-dimensional system of Argon atoms interacting via the Lennard-Jones (LJ) potential and analyzes the microscopic dynamics by computing the Velocity Autocorrelation Function (VACF) and its Fourier-transformed spectrum.

---

## Configuration Parameters (passed via `params` dictionary):
- `dimensions`: Simulation dimensionality (fixed to 3D).
- `n_particles`: Number of Argon atoms (default: 864, matching Rahman's study).
- `density`: Mass density of Argon ($1374 \, \mathrm{kg/m^3}$).
- `dt`: Time step for integration ($2 \times 10^{-15}\,\mathrm{s}$).
- `steps`: Total number of simulation steps (default: 1000).
- `temperature`: Target temperature ($94.4 \, \mathrm{K}$).
- `sigma`, `epsilon`: Lennard-Jones potential parameters for Argon ($\sigma = 3.4 \times 10^{-10}\,\mathrm{m}$, $\epsilon = 1.656 \times 10^{-21}\,\mathrm{J}$).
- `rcutoff`: Cutoff distance for LJ interactions ($2.5\sigma$).
- `tau_ber`: Relaxation time for Berendsen thermostat.
- `gamma_langevin`: Friction constant for Langevin thermostat.
- `thermostat_type`: Thermostat type (`"berendsen"` or `"langevin"`).

---

## Key Functions and Return Values:
- `compute_lj_force(r, sigma, epsilon, rcutoff)`: Calculates the Lennard-Jones force.
- `compute_lj_potential(r, sigma, epsilon, rcutoff)`: Computes the shifted Lennard-Jones potential.
- `build_linked_cells(positions, box_size, rcutoff)`: Constructs neighbor lists using the linked-cell algorithm.
- `compute_forces_lca(...)`: Computes forces using the linked-cell method with periodic boundary conditions.
- `create_lattice(n_particles, box_size, dimensions)`: Initializes atom positions on a lattice with random perturbations.
- `initialize_velocities(n_particles, dimensions, target_temp)`: Samples initial velocities from a Maxwell-Boltzmann distribution.
- `apply_berendsen_thermostat(velocities, target_temp, current_temp, dt, tau)`: Applies Berendsen thermostat.
- `apply_langevin_thermostat(velocities, gamma, dt, target_temp)`: Applies Langevin thermostat with stochastic dynamics.
- `compute_vacf(velocities_over_time)`: Computes the normalized velocity autocorrelation function (VACF) over time.
- `compute_vacf_spectrum(vacf, dt)`: Computes the Fourier spectrum from the VACF.
- `run_simulation(params, compute_rdf_flag=False, rdf_sample_steps=1000, n_bins=100)`: Main function that runs the molecular dynamics simulation and collects VACF data.

---

## Simulation Workflow:
- Create a cubic simulation box and place 864 Argon atoms.
- Integrate the equations of motion using the velocity-Verlet scheme.
- Regulate temperature using the Langevin thermostat.
- Store particle velocities at each timestep.
- Compute the Velocity Autocorrelation Function (VACF) based on initial velocities.
- Normalize the VACF to its initial value ($\mathrm{VACF}(0) = 1$).
- Perform a Fourier transform of the VACF to obtain the vibrational spectrum.
- Normalize the spectrum to have a maximum value of 1.

---

## Output:
- **Normalized VACF** plot: shows how particle velocities decorrelate over time (measured in picoseconds).
- **Normalized Spectrum** plot: shows the vibrational modes (peaks at characteristic frequencies in THz).

---

## Parameters:
| Parameter         | Value                      |
|-------------------|-----------------------------|
| dimensions        | 3                           |
| n_particles       | 864                         |
| density           | 1374 $\mathrm{kg/m^3}$       |
| dt                | $2 \times 10^{-15}\,\mathrm{s}$ |
| steps             | 1000                        |
| temperature       | 94.4 $\mathrm{K}$            |
| sigma             | $3.4 \times 10^{-10}\,\mathrm{m}$ |
| epsilon           | $1.656 \times 10^{-21}\,\mathrm{J}$ |
| rcutoff           | $2.5\sigma$                  |
| tau_ber           | $1 \times 10^{-13}\,\mathrm{s}$ |
| gamma_langevin    | $1 \times 10^{13}\,\mathrm{s^{-1}}$ |

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm 


sigma = 3.4e-10  # m
epsilon = 1.656e-21  # J
mass_argon = 39.95 * 1.66054e-27  # kg
density = 1374  # kg/m³


def compute_lj_force(r, sigma, epsilon, rcutoff):
    if r >= rcutoff or r < 1e-12:
        return 0.0
    inv_r = sigma / r
    inv_r6 = inv_r ** 6
    inv_r12 = inv_r6 ** 2
    return 24 * epsilon * (2 * inv_r12 - inv_r6) / r

def compute_lj_potential(r, sigma, epsilon, rcutoff):
    if r >= rcutoff:
        return 0.0
    inv_r = sigma / r
    inv_r6 = inv_r ** 6
    inv_r12 = inv_r6 ** 2
    pot = 4 * epsilon * (inv_r12 - inv_r6)
    inv_rcut = sigma / rcutoff
    shift = 4 * epsilon * (inv_rcut ** 12 - inv_rcut ** 6)
    return pot - shift


def build_linked_cells(positions, box_size, rcutoff):
    n_particles, dim = positions.shape
    lc = max(1, int(np.floor(box_size / rcutoff)))
    lc_dim = [lc] * dim
    rc = box_size / lc
    EMPTY = -1
    head = [EMPTY] * (lc ** dim)
    lscl = [EMPTY] * n_particles

    for i in range(n_particles):
        mc = [int(positions[i][a] / rc) for a in range(dim)]
        mc = [min(max(0, idx), lc - 1) for idx in mc]
        if dim == 2:
            c_index = mc[0] * lc_dim[1] + mc[1]
        else:
            c_index = mc[0] * lc_dim[1] * lc_dim[2] + mc[1] * lc_dim[2] + mc[2]
        lscl[i] = head[c_index]
        head[c_index] = i

    return head, lscl, lc_dim

def compute_forces_lca(positions, box_size, rcutoff, sigma, epsilon, use_pbc=True):
    n_particles, dim = positions.shape
    head, lscl, lc_dim = build_linked_cells(positions, box_size, rcutoff)
    EMPTY = -1
    forces = np.zeros_like(positions)
    potential_energy = 0.0
    neighbor_offsets = np.array(np.meshgrid(*[[-1, 0, 1]] * dim)).T.reshape(-1, dim)

    for mc in np.ndindex(*lc_dim):
        if dim == 2:
            c_index = mc[0] * lc_dim[1] + mc[1]
        else:
            c_index = mc[0] * lc_dim[1] * lc_dim[2] + mc[1] * lc_dim[2] + mc[2]
        i = head[c_index]
        while i != EMPTY:
            pos_i = positions[i]
            for offset in neighbor_offsets:
                mc1 = np.array(mc) + offset
                rshift = np.zeros(dim)
                valid_cell = True
                for a in range(dim):
                    if use_pbc:
                        if mc1[a] < 0:
                            mc1[a] += lc_dim[a]
                            rshift[a] = -box_size
                        elif mc1[a] >= lc_dim[a]:
                            mc1[a] -= lc_dim[a]
                            rshift[a] = box_size
                    else:
                        if mc1[a] < 0 or mc1[a] >= lc_dim[a]:
                            valid_cell = False
                            break
                if not valid_cell:
                    continue
                if dim == 2:
                    c1 = mc1[0] * lc_dim[1] + mc1[1]
                else:
                    c1 = mc1[0] * lc_dim[1] * lc_dim[2] + mc1[1] * lc_dim[2] + mc1[2]
                j = head[c1]
                while j != EMPTY:
                    if j > i:
                        pos_j = positions[j] + rshift
                        r_ij = pos_i - pos_j
                        dist = np.linalg.norm(r_ij)
                        if dist < rcutoff and dist > 1e-12:
                            f_mag = compute_lj_force(dist, sigma, epsilon, rcutoff)
                            fij = f_mag * (r_ij / dist)
                            forces[i] += fij
                            forces[j] -= fij
                            potential_energy += compute_lj_potential(dist, sigma, epsilon, rcutoff)
                    j = lscl[j]
            i = lscl[i]
    return forces, potential_energy


def apply_berendsen_thermostat(velocities, target_temp, current_temp, dt, tau):
    lambda_factor = np.sqrt(1 + dt / tau * (target_temp / current_temp - 1))
    return velocities * lambda_factor

def apply_langevin_thermostat(velocities, gamma, dt, target_temp):
    kb = 1.380649e-23  
    noise = np.random.normal(0, 1, size=velocities.shape)
    c1 = np.exp(-gamma * dt)
    c2 = np.sqrt((1 - c1**2) * kb * target_temp / mass_argon)
    return c1 * velocities + c2 * noise


def create_lattice(n_particles, box_size, dimensions):
    n_side = int(np.ceil(n_particles ** (1 / dimensions)))
    spacing = box_size / n_side
    positions = []
    for indices in np.ndindex(*([n_side] * dimensions)):
        if len(positions) < n_particles:
            pos = [(i + 0.5) * spacing for i in indices]
            noise = np.random.uniform(-0.05, 0.05, size=dimensions) * spacing
            positions.append(np.array(pos) + noise)
    return np.array(positions)

def initialize_velocities(n_particles, dimensions, target_temp):
    kb = 1.380649e-23  
    velocities = np.random.normal(0, 1, size=(n_particles, dimensions))
    velocities -= np.mean(velocities, axis=0)
    ke = 0.5 * mass_argon * np.sum(velocities**2)
    dof = n_particles * dimensions
    scale = np.sqrt(target_temp * kb * dof / (2 * ke))
    velocities *= scale
    return velocities


def compute_vacf(velocities_over_time):
    n_steps, n_particles, dimensions = velocities_over_time.shape
    vacf = np.zeros(n_steps)
    v0 = velocities_over_time[0]
    for t in range(n_steps):
        vt = velocities_over_time[t]
        dot_products = np.sum(v0 * vt, axis=1)
        vacf[t] = np.mean(dot_products)
    return vacf

def compute_vacf_spectrum(vacf, dt):
    vacf_fft = np.fft.fft(vacf)
    freqs = np.fft.fftfreq(len(vacf), d=dt)
    spectrum = np.real(vacf_fft)
    return freqs[:len(freqs)//2], spectrum[:len(spectrum)//2]

# --- Simulation loop ---
def run_simulation(params, compute_rdf_flag=False, rdf_sample_steps=1000, n_bins=100):
    box_size = (params['n_particles'] * mass_argon / params['density']) ** (1 / params['dimensions'])
    positions = create_lattice(params['n_particles'], box_size, params['dimensions'])
    velocities = initialize_velocities(params['n_particles'], params['dimensions'], params['temperature'])
    dof = params['n_particles'] * params['dimensions']

    temp_list = []
    rdf_accum = np.zeros(n_bins)
    rdf_count = 0
    velocities_over_time = []

    for step in tqdm(range(params['steps']), desc="Running Simulation"):
        forces, _ = compute_forces_lca(positions, box_size, params['rcutoff'],
                                       params['sigma'], params['epsilon'], use_pbc=True)
        velocities += 0.5 * forces / mass_argon * params['dt']
        positions += velocities * params['dt']
        positions %= box_size
        forces, _ = compute_forces_lca(positions, box_size, params['rcutoff'],
                                       params['sigma'], params['epsilon'], use_pbc=True)
        velocities += 0.5 * forces / mass_argon * params['dt']

        kinetic = 0.5 * mass_argon * np.sum(velocities ** 2)
        temp = 2 * kinetic / (dof * 1.380649e-23)
        temp_list.append(temp)

        velocities_over_time.append(velocities.copy())

        if params['thermostat_type'] == 'berendsen':
            velocities = apply_berendsen_thermostat(velocities, params['temperature'], temp, params['dt'], params['tau_ber'])
        elif params['thermostat_type'] == 'langevin':
            velocities = apply_langevin_thermostat(velocities, params['gamma_langevin'], params['dt'], params['temperature'])

    velocities_over_time = np.array(velocities_over_time)
    vacf = compute_vacf(velocities_over_time)
    freqs, spectrum = compute_vacf_spectrum(vacf, params['dt'])
    
    return temp_list, vacf, freqs, spectrum

params = {
    'dimensions': 3,
    'n_particles': 864,
    'density': 1374,
    'dt': 2e-15,
    'steps': 2000,
    'temperature': 94.4,
    'sigma': sigma,
    'epsilon': epsilon,
    'rcutoff': 2.5 * sigma,
    'tau_ber': 1e-13,
    'gamma_langevin': 1e13,
    'thermostat_type': 'langevin'
}

time_array = np.arange(len(vacf)) * params['dt']
vacf_normalized = vacf / vacf[0]
spectrum_normalized = spectrum / np.max(np.abs(spectrum))


plt.figure(figsize=(8, 5))
plt.plot(time_array * 1e12, vacf_normalized)
plt.xlabel('Time (ps)')
plt.ylabel('Normalized VACF')
plt.title('Normalized Velocity Autocorrelation Function (VACF)')
plt.grid()
plt.tight_layout()
plt.show()
plt.figure(figsize=(8, 5))
plt.plot(freqs * 1e-12, spectrum_normalized)
plt.xlabel('Frequency (THz)')
plt.ylabel('Normalized Spectrum')
plt.title('Normalized Velocity Spectrum')
plt.grid()
plt.tight_layout()
plt.show()

NameError: name 'vacf' is not defined